In [106]:
#libraries we need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import date
pd.options.mode.chained_assignment = None
import h2o
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

#libraries we need
# !pip install h2o

from scipy.special import expit
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch
from sklearn import preprocessing

from sklearn.model_selection import train_test_split
from h2o.estimators import H2OGradientBoostingEstimator
SEED  = 1111   # global random seed for better reproducibility

from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import pydotplus

# conda install -c conda-forge pydotplus

h2o.init(max_mem_size='24G', nthreads=4) # start h2o with plenty of memory and threads
h2o.remove_all()                         # clears h2o memory
h2o.no_progress() 

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,3 hours 36 mins
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.5
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_zhangmaojn_h2kfdg
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,21.15 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [107]:
train = pd.read_csv('adult.data.txt',
                    names=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','class'])

In [108]:
test = pd.read_csv('adult.test.txt',
                    names=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','class'])

In [109]:
train.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [110]:
train.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
class             object
dtype: object

In [111]:
train.shape

(32561, 15)

In [112]:
test

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16276,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
16277,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
16278,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
16279,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


In [113]:
## USE THE ABOVE TO REMOVE THE '?'

Replace all entries that contains ? with Null

In [114]:
train=train.replace(' ?', np.nan)

test=test.replace(' ?', np.nan)

Fill Null with 'None' type

In [115]:
train.update(train[['native-country','occupation','workclass']].fillna('None'))
test.update(test[['native-country','occupation','workclass']].fillna('None'))

In [116]:
train.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64

In [117]:
test.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
class             0
dtype: int64

In [118]:
train.to_csv('train.csv',index=False)

In [119]:
test.to_csv('test.csv',index=False)

## GLM

GLM can produce two categories of models: classification and regression.

Logistic regression is the GLM performing binary classification.

In [26]:
hf=h2o.H2OFrame(train)
gf=h2o.H2OFrame(test)

In [27]:
response_col = 'class'
predictors = list(train.columns.values)
predictors.remove('class')

WE need the value of alpha also using grid search cv; 
refer the house pricing dataset and the link below for the same. 
Finish this model and we will do XGBoost tomorrow.  

In [28]:
glm_model = H2OGeneralizedLinearEstimator(family= "binomial",
                                          lambda_ = 0,
                                          compute_p_values = True,remove_collinear_columns=True)
glm_model.train(predictors, response_col, training_frame= hf)

In [29]:
glm_model.predict(hf)

predict,<=50K,>50K,StdErr
<=50K,0.870935,0.129065,0.1293
>50K,0.597296,0.402704,0.0954489
<=50K,0.971805,0.0281947,0.148196
<=50K,0.917899,0.0821005,0.203921
>50K,0.327233,0.672767,0.32746
>50K,0.173244,0.826756,0.103964
<=50K,0.998289,0.00171141,0.546186
>50K,0.604937,0.395063,0.0778746
>50K,0.0926695,0.907331,0.172349
>50K,0.0830572,0.916943,0.0768653


#### GET IT WORKING ON THE TEST SET GET THE ERROR ON THE TEST SET; USE THE LINK BELOW

 https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/glm.html?highlight=glm

### Note: Quick change to Markdown cell! 
<------- Clike on the grean bar on the left side    

Once it turn to blue

press 'm' on the keyboard

That's when the magic happens! Try it. 

'm' ---Markdown 

'y' ---Code

These can only be down after you click on the left side bar ;)

#### Link where i got this example: 
    
https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/algo-params/lambda.html
    

In [123]:
# set the predictor names and the response column name

hf=h2o.H2OFrame(train)
gf=h2o.H2OFrame(test)
response_col = 'class'
predictors = list(train.columns.values)
predictors.remove('class')


# try using the `lambda_` parameter:
# initialize your estimator
class_glm = H2OGeneralizedLinearEstimator(family = 'binomial', lambda_ = .0001)

# then train your model
class_glm.train(x = predictors, y = response_col, training_frame = hf, validation_frame = gf)

# print the auc for the validation data
print(class_glm.auc(valid=True))

0.9057526281750908


In [121]:
# Example of values to grid over for `lambda`
# import Grid Search

# select the values for lambda_ to grid over
hyper_params = {'lambda': [1, 0.5, 0.1, 0.01, 0.001, 0.0001, 0.00001, 0]}

# this example uses cartesian grid search because the search space is small
# and we want to see the performance of all models. 
#For a larger search space use random grid search instead: {'strategy': "RandomDiscrete"}
# initialize the glm estimator
class_glm_2 = H2OGeneralizedLinearEstimator(family = 'binomial')

# build grid search with previously made GLM and hyperparameters
grid = H2OGridSearch(model = class_glm_2, hyper_params = hyper_params,
                     search_criteria = {'strategy': "Cartesian"})

# train using the grid
grid.train(x = predictors, y = response_col, training_frame = hf, validation_frame = gf)

# sort the grid models by decreasing AUC
grid_table = grid.get_grid(sort_by = 'auc', decreasing = True)
print(grid_table)

best = grid_table.models[0]
print(best.actual_params['lambda'])

       lambda  \
0    [1.0E-4]   
1     [0.001]   
2    [1.0E-5]   
3       [0.0]   
4      [0.01]   
5       [0.1]   
6       [1.0]   
7       [0.5]   

                                                               model_ids  \
0  Grid_GLM_Key_Frame__upload_97a58ab56c1787f886748d51e6424c10.hex_mo...   
1  Grid_GLM_Key_Frame__upload_97a58ab56c1787f886748d51e6424c10.hex_mo...   
2  Grid_GLM_Key_Frame__upload_97a58ab56c1787f886748d51e6424c10.hex_mo...   
3  Grid_GLM_Key_Frame__upload_97a58ab56c1787f886748d51e6424c10.hex_mo...   
4  Grid_GLM_Key_Frame__upload_97a58ab56c1787f886748d51e6424c10.hex_mo...   
5  Grid_GLM_Key_Frame__upload_97a58ab56c1787f886748d51e6424c10.hex_mo...   
6  Grid_GLM_Key_Frame__upload_97a58ab56c1787f886748d51e6424c10.hex_mo...   
7  Grid_GLM_Key_Frame__upload_97a58ab56c1787f886748d51e6424c10.hex_mo...   

                  auc  
0  0.9057526281750908  
1  0.9053984829276566  
2  0.9050907255429743  
3  0.9045369357999089  
4  0.8961385580473481  
5  0.867214047629